In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import tensorflow as tf
import os
import shutil
import cv2 as cv
from PIL import Image
import keras
from keras import models
from sklearn.metrics import accuracy_score, confusion_matrix
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
# from keras.utils import to_categorical
from keras.models import Sequential
from keras.models import Model, load_model
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, \
    AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, Dropout, GlobalAveragePooling2D, Resizing, LeakyReLU
from keras.applications.efficientnet import EfficientNetB0
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras import initializers, layers
from tensorflow.keras import optimizers
from keras.initializers import glorot_uniform
from keras.applications.vgg16 import VGG16
from keras.applications.vgg19 import VGG19
from keras.utils.layer_utils import get_source_inputs
from keras_vggface.vggface import VGGFace
from sklearn.model_selection import train_test_split
from keras.applications.resnet import ResNet50
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.utils import to_categorical

from mtcnn.mtcnn import MTCNN


In [2]:
def mtcnn(mtcnn_dir, training_dir):

    def affineMatrix(lmks, scale=2.5):
        nose = np.array(lmks['nose'], dtype=np.float32)
        left_eye = np.array(lmks['left_eye'], dtype=np.float32)
        right_eye = np.array(lmks['right_eye'], dtype=np.float32)
        eye_width = right_eye - left_eye
        angle = np.arctan2(eye_width[1], eye_width[0])
        center = nose
        alpha = np.cos(angle)
        beta = np.sin(angle)
        w = np.sqrt(np.sum(eye_width ** 2)) * scale
        m = [[alpha, beta, -alpha * center[0] - beta * center[1] + w * 0.5],
             [-beta, alpha, beta * center[0] - alpha * center[1] + w * 0.5]]
        return np.array(m), (int(w), int(w))  # （affine matrix, target size)

    detector = MTCNN(steps_threshold=[0.0, 0.0, 0.0])
    for i in training_dir:
        print("processing ", i)
        img = cv.imread(training_dir + i)
        faces = detector.detect_faces(cv.cvtColor(img, cv.COLOR_BGR2RGB))
        face = max(faces, key=lambda x: x['confidence'])
        mat, size = affineMatrix(face['keypoints'])

        tmp = cv.warpAffine(img, mat, size)
        cv.imwrite(mtcnn_dir + i, tmp)

        img = Image.open(mtcnn_dir + i)
        new_img = img.resize((256, 256))
        new_img.save(mtcnn_dir + i)

In [3]:
def preprocessing(deepfake_dir, face2face_dir, real_dir, training_dir, Re_save=False, mt_cnn=False):
    deepfake_img = os.listdir(deepfake_dir)
    face2face_img = os.listdir(face2face_dir)
    real_img = os.listdir(real_dir)

    if Re_save:
        if not os.path.exists(training_dir):
            os.makedirs(training_dir)

        for filename in deepfake_img:
            print("Deepfake")
            original = deepfake_dir + filename
            target = training_dir + 'deepfake_' + filename
            shutil.copyfile(original, target)

        for filename in face2face_img:
            print("Face2Face")
            original = face2face_dir + filename
            target = training_dir + 'face2face_' + filename
            shutil.copyfile(original, target)

        for filename in real_img:
            print("Real")
            original = real_dir + filename
            target = training_dir + 'real_' + filename
            shutil.copyfile(original, target)

    if mt_cnn:
        mtcnn_dir = './testing/'
        mtcnn(mtcnn_dir, training_dir)
        training_img = os.listdir("./testing/")
    else:
        training_img = os.listdir("./training_data/")

    categories = []
    for i in training_img:
        tmp = i.split("_")[0]
        if tmp == "deepfake":
            categories.append("fake")
        elif tmp == "face2face":
            categories.append("fake")
        else:
            categories.append("real")

    df = pd.DataFrame({
        'filename': training_img,
        'category': categories
    })

    return df

In [4]:
def generate_data_set(df, batch_size):
    df["category"] = df["category"].replace({0: 'fake', 1: 'real'})
    train_df, validate_df = train_test_split(df, test_size=0.20, random_state=None, stratify=df["category"])

    train_df = train_df.reset_index(drop=True)
    validate_df = validate_df.reset_index(drop=True)

    total_train = train_df.shape[0]
    total_validate = validate_df.shape[0]

    # Training Set
    train_datagen = ImageDataGenerator(
        rotation_range=15,
        rescale=1. / 255,
        shear_range=0.1,
        zoom_range=0.2,
        horizontal_flip=True,
        width_shift_range=0.1,
        height_shift_range=0.1
    )

    train_generator = train_datagen.flow_from_dataframe(
        train_df,
        "./training_data/",
        x_col='filename',
        y_col='category',
        target_size=IMAGE_SIZE,  # Default : 299, 299
        class_mode='binary',
        batch_size=batch_size
    )

    print(train_generator.class_indices)

    # =======================================================
    # Validation Set
    validation_datagen = ImageDataGenerator(rescale=1. / 255)
    validation_generator = validation_datagen.flow_from_dataframe(
        validate_df,
        "./training_data/",
        x_col='filename',
        y_col='category',
        target_size=IMAGE_SIZE,
        class_mode='binary',
        batch_size=batch_size
    )

    print(validation_generator.class_indices)

    return train_generator, validation_generator, total_train, total_validate


In [5]:
def model_training(model, train_data, validate_data, batch_size, num_train, num_validation):
    # Prevent the model being overfit
    earlystop = EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=2,
                              verbose=0, mode='auto')

    # Improve the learning rate
    learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy',
                                                patience=2,
                                                verbose=1,
                                                mode='auto')
                                                # factor=0.5,
                                                # min_lr=0.00001)

    callbacks = [earlystop, learning_rate_reduction]

    total_train = num_train
    total_validate = num_validation
    batch_size = batch_size

    if TRAIN:
        epochs = 3 if FAST_RUN else 30

        history = model.fit(
            train_data,
            epochs=epochs,
            validation_data=validate_data,
            validation_steps=total_validate // batch_size,
            steps_per_epoch=total_train // batch_size,
            callbacks=callbacks
        )

        model.save_weights("./testing.h5")

        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 12))
        ax1.plot(history.history['loss'], color='b', label="Training loss")
        ax1.plot(history.history['val_loss'], color='r', label="validation loss")
        ax1.set_xticks(np.arange(1, epochs, 1))
        ax1.set_yticks(np.arange(0, 1, 0.1))
        legend = ax1.legend(loc='best', shadow=True)

        ax2.plot(history.history['accuracy'], color='b', label="Training accuracy")
        ax2.plot(history.history['val_accuracy'], color='r', label="Validation accuracy")
        ax2.set_xticks(np.arange(1, epochs, 1))

        legend = ax2.legend(loc='best', shadow=True)
        plt.tight_layout()
        plt.savefig("./test.png")
        plt.show()

In [9]:
def create_EfficientNet():
    model = EfficientNetB0(weights="imagenet", include_top=False, input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, 3))
    print('Model loaded.')

    # build a classifier model to put on top of the convolutional model
    top_model = Sequential()  # Determines the type of top model

    top_model.add(model)

    # formats the input for the classifier to the convolutional base output
    top_model.add(Flatten(input_shape=model.output_shape[1:]))

    # Condenses the input from flatten down to 256 nodes
    top_model.add(Dense(256, activation='relu'))

    top_model.add(Dropout(0.5))

    # Condenses the remaining input down into 2 categories
    top_model.add(Dense(1, activation='softmax', name='predictions'))

    top_model.compile(loss=loss_function,
                      optimizer=optimizers.SGD(lr=learning_rate, momentum=momentum, decay=1e-6, nesterov=False),
                      # learning rate(lr) and momentum are Core Variables
                      # SGD options to consider: Nesterov, learning rate decay
                      metrics=[metrics])

    # top_model.compile(loss=loss_function, optimizer='Adam', metrics=['accuracy'])
    return top_model

In [10]:
def create_VGG16():
    model = VGG16(weights="imagenet", include_top=False, input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, 3))
    # model = ResNet50(weights="imagenet", include_top=False, input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, 3))
    print('Model loaded.')

    # build a classifier model to put on top of the convolutional model
    top_model = Sequential()  # Determines the type of top model

    top_model.add(model)

    # formats the input for the classifier to the convolutional base output
    top_model.add(Flatten(input_shape=model.output_shape[1:]))

    # Condenses the input from flatten down to 256 nodes
    top_model.add(Dense(256, activation='relu'))

    top_model.add(Dropout(0.5))

    # Condenses the remaining input down into 2 categories
    top_model.add(Dense(1, activation='softmax', name='predictions'))

    top_model.compile(loss=loss_function,
                      optimizer=optimizers.SGD(lr=learning_rate, momentum=momentum, decay=1e-6, nesterov=False),
                      # learning rate(lr) and momentum are Core Variables
                      # SGD options to consider: Nesterov, learning rate decay
                      metrics=[metrics])

    # top_model.compile(loss=loss_function, optimizer='Adam', metrics=['accuracy'])
    return top_model

In [11]:
def create_ResNet50():
    model = ResNet50(weights="imagenet", include_top=False, input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, 3))
    print('Model loaded.')

    # build a classifier model to put on top of the convolutional model
    top_model = Sequential()  # Determines the type of top model

    top_model.add(model)

    # formats the input for the classifier to the convolutional base output
    top_model.add(Flatten(input_shape=model.output_shape[1:]))

    # Condenses the input from flatten down to 256 nodes
    top_model.add(Dense(256, activation='relu'))

    top_model.add(Dropout(0.5))

    top_model.add(Dense(128, activation='relu'))

    # Condenses the remaining input down into 2 categories
    top_model.add(Dense(1, activation='softmax', name='predictions'))

    top_model.compile(loss=loss_function,
                      optimizer=optimizers.SGD(lr=learning_rate, momentum=momentum, decay=1e-6, nesterov=False),
                      # learning rate(lr) and momentum are Core Variables
                      # SGD options to consider: Nesterov, learning rate decay
                      metrics=[metrics])

    # top_model.compile(loss=loss_function, optimizer='Adam', metrics=['accuracy'])
    return top_model

In [12]:
def create_mesonet():
    x = Input(shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS))
    x1 = Resizing(224, 224, interpolation='bilinear', crop_to_aspect_ratio=False)(x)
    x1 = Conv2D(8, (3, 3), padding='same', activation='relu')(x1)
    x1 = BatchNormalization()(x1)
    x1 = MaxPooling2D(pool_size=(2, 2), padding='same')(x1)

    x2 = Conv2D(8, (5, 5), padding='same', activation='relu')(x1)
    x2 = BatchNormalization()(x2)
    x2 = MaxPooling2D(pool_size=(2, 2), padding='same')(x2)

    x3 = Conv2D(16, (5, 5), padding='same', activation='relu')(x2)
    x3 = BatchNormalization()(x3)
    x3 = MaxPooling2D(pool_size=(2, 2), padding='same')(x3)

    x4 = Conv2D(16, (5, 5), padding='same', activation='relu')(x3)
    x4 = BatchNormalization()(x4)
    x4 = MaxPooling2D(pool_size=(4, 4), padding='same')(x4)

    y = Flatten()(x4)
    y = Dropout(0.5)(y)
    y = Dense(16)(y)
    y = LeakyReLU(alpha=0.1)(y)
    y = Dropout(0.5)(y)
    y = Dense(1, activation='sigmoid')(y)

    model = Model(inputs=x, outputs=y)
    model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(0.0002), metrics=['accuracy'])

    return model

In [15]:
def create_model():
    model = Sequential()
    model.add(Conv2D(256, (3, 3), activation='relu', input_shape=(IMAGE_WIDTH, IMAGE_HEIGHT, IMAGE_CHANNELS)))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(2, activation='relu'))
    model.add(Dense(128, activation='relu'))

    model.add(Dense(1, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    return model

In [16]:

TRAIN = True
Re_save_data = False
use_mt_cnn = False
FAST_RUN = False
batch_size = 20
loss_function = 'categorical_crossentropy'
learning_rate = 0.01
momentum = 0.9
metrics = 'accuracy'

IMAGE_WIDTH = 299
IMAGE_HEIGHT = 299
IMAGE_SIZE = (IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS = 3  # R, G, B

deepfake_dir = "./fake_deepfake/"
face2face_dir = "./fake_face2face/"
real_dir = "./real/"
training_dir = "./training_data/"


# model = create_VGG16()
# model = create_ResNet50()
# model = create_EfficientNet()
# model = create_mesonet()
# model = test_model()
model = create_model()

model.summary()

df = preprocessing(deepfake_dir, face2face_dir, real_dir, training_dir, Re_save_data, use_mt_cnn)

train_set, validation_set, num_train, num_validation = generate_data_set(df, batch_size)

model_training(model, train_set, validation_set, batch_size, num_train, num_validation)
exit()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 297, 297, 256)     7168      
                                                                 
 batch_normalization_4 (Batc  (None, 297, 297, 256)    1024      
 hNormalization)                                                 
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 148, 148, 256)    0         
 2D)                                                             
                                                                 
 dropout_2 (Dropout)         (None, 148, 148, 256)     0         
                                                                 
 conv2d_5 (Conv2D)           (None, 146, 146, 512)     1180160   
                                                                 
 batch_normalization_5 (Batc  (None, 146, 146, 512)    2

ResourceExhaustedError:  OOM when allocating tensor with shape[20,512,146,146] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node sequential/conv2d_5/Relu
 (defined at C:\Users\Marco\anaconda3\envs\ML_tensorflow\lib\site-packages\keras\backend.py:4867)
]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_7691]

Errors may have originated from an input operation.
Input Source operations connected to node sequential/conv2d_5/Relu:
In[0] sequential/conv2d_5/BiasAdd (defined at C:\Users\Marco\anaconda3\envs\ML_tensorflow\lib\site-packages\keras\layers\convolutional.py:264)

Operation defined at: (most recent call last)
>>>   File "C:\Users\Marco\anaconda3\envs\ML_tensorflow\lib\runpy.py", line 197, in _run_module_as_main
>>>     return _run_code(code, main_globals, 